In [1]:
import os
import pandas as pd
import numpy as np
import re

# Use the current directory where the notebook is located
base_path = os.getcwd()

# Extract case and stop-to-stop info from the folder name
folder_name = os.path.basename(base_path)
case_match = re.search(r"Case_(.*?)_StopToStop_(\d+)", folder_name)
case_id = case_match.group(1) if case_match else "NA"
stop_to_stop = case_match.group(2) if case_match else "NA"

# Output will be saved in the same folder as the notebook
output_dir = base_path

# Loop through all Kappa_* folders
for kappa_folder in sorted(os.listdir(base_path)):
    if not kappa_folder.startswith("Kappa_"):
        continue
    kappa_val = kappa_folder.split("_")[1]
    kappa_path = os.path.join(base_path, kappa_folder)

    results = []

    # Loop through all Density_* folders
    for density_folder in sorted(os.listdir(kappa_path)):
        if not density_folder.startswith("Density_"):
            continue
        arrival_rate = density_folder.split("_")[1]
        density_path = os.path.join(kappa_path, density_folder)

        # Loop through all PassengerRate_* folders
        for passenger_rate_folder in sorted(os.listdir(density_path)):
            if not passenger_rate_folder.startswith("PassengerRate_"):
                continue
            passenger_rate_path = os.path.join(density_path, passenger_rate_folder)
            timestep_path = os.path.join(passenger_rate_path, "TimestepSummary")

            if not os.path.exists(timestep_path):
                print(f"⚠️ Skipping missing: {timestep_path}")
                continue

            # Collectors for stats
            densities = []
            truck_fractions = []
            max_throughputs = []
            max_passenger_throughputs = []
            max_jeep_throughputs = []
            max_truck_throughputs = []

            # Process up to 50 trial CSVs
            for file in sorted(os.listdir(timestep_path))[:50]:
                if not file.startswith("Trial_") or not file.endswith(".csv"):
                    continue
                file_path = os.path.join(timestep_path, file)

                try:
                    df = pd.read_csv(file_path)
                    df_filtered = df[df['Timestep'] > 6999]
                    if df_filtered.empty:
                        continue

                    # Get values at t = 6999
                    t0 = df.loc[df['Timestep'] == 6999, 'Throughput'].values
                    j0 = df.loc[df['Timestep'] == 6999, 'Jeep Throughput'].values
                    tr0 = df.loc[df['Timestep'] == 6999, 'Truck Throughput'].values

                    t0 = t0[0] if len(t0) > 0 else 0
                    j0 = j0[0] if len(j0) > 0 else 0
                    tr0 = tr0[0] if len(tr0) > 0 else 0

                    # Throughput deltas
                    max_throughput = df_filtered["Throughput"].max() - t0
                    max_passenger_throughput = df_filtered["Passenger Throughput"].max() - t0
                    max_jeep_throughput = df_filtered["Jeep Throughput"].max() - j0
                    max_truck_throughput = df_filtered["Truck Throughput"].max() - tr0

                    # Get density & truck fraction
                    actual_density = df_filtered["Actual Density"].iloc[0]
                    actual_truck_fraction = df_filtered.get("Actual Truck Fraction", pd.Series([np.nan])).iloc[0]

                    # Append to lists
                    densities.append(actual_density)
                    truck_fractions.append(actual_truck_fraction)
                    max_throughputs.append(max_throughput)
                    max_passenger_throughputs.append(max_passenger_throughput)
                    max_jeep_throughputs.append(max_jeep_throughput)
                    max_truck_throughputs.append(max_truck_throughput)

                except Exception as e:
                    print(f"❌ Error reading {file_path}: {e}")
                    continue

            # Skip if no data
            if not densities:
                continue

            # Compute and store stats
            row = [
                kappa_folder, density_folder, passenger_rate_folder,
                np.mean(densities), np.std(densities, ddof=1), np.median(densities),
                np.mean(truck_fractions), np.std(truck_fractions, ddof=1), np.median(truck_fractions),
                np.mean(max_throughputs), np.std(max_throughputs, ddof=1), np.median(max_throughputs),
                np.mean(max_passenger_throughputs), np.std(max_passenger_throughputs, ddof=1), np.median(max_passenger_throughputs),
                np.mean(max_jeep_throughputs), np.std(max_jeep_throughputs, ddof=1), np.median(max_jeep_throughputs),
                np.mean(max_truck_throughputs), np.std(max_truck_throughputs, ddof=1), np.median(max_truck_throughputs)
            ]
            results.append(row)

    # Save to CSV
    if results:
        df_results = pd.DataFrame(results, columns=[
            "Kappa", "Density", "PassengerRate",
            "Mean Density", "Std Density", "Median Density",
            "Mean Truck Fraction", "Std Truck Fraction", "Median Truck Fraction",
            "Mean Max Throughput", "Std Max Throughput", "Median Max Throughput",
            "Mean Max Passenger Throughput", "Std Max Passenger Throughput", "Median Max Passenger Throughput",
            "Mean Max Jeepney Throughput", "Std Max Jeepney Throughput", "Median Max Jeepney Throughput",
            "Mean Max Truck Throughput", "Std Max Truck Throughput", "Median Max Truck Throughput"
        ])

        output_filename = f"Flow_Summary_K{kappa_val}_R{arrival_rate}_S{stop_to_stop}_Case{case_id}.csv"
        output_path = os.path.join(output_dir, output_filename)
        df_results.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}")
